# Восстановление золота из руды

Подготовьте прототип модели машинного обучения для «Цифры». Компания разрабатывает решения для эффективной работы промышленных предприятий.

Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. Используйте данные с параметрами добычи и очистки. 

Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.

Вам нужно:

1. Подготовить данные;
2. Провести исследовательский анализ данных;
3. Построить и обучить модель.

Чтобы выполнить проект, обращайтесь к библиотекам *pandas*, *matplotlib* и *sklearn.* Вам поможет их документация.

## Подготовка данных

### Откройте файлы и изучите их

In [ ]:
import pandas as pd

import numpy as np
from numpy.random import RandomState

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.dummy import DummyRegressor

import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from scipy.stats import ks_2samp
from scipy.stats import iqr

import warnings
warnings.filterwarnings("ignore")


from tqdm import tqdm

TARGETS = ['rougher.output.recovery', 'final.output.recovery']

In [ ]:
data_train = pd.read_csv("/datasets/gold_recovery_train_new.csv")
data_test = pd.read_csv("/datasets/gold_recovery_test_new.csv")
data_full = pd.read_csv("/datasets/gold_recovery_full_new.csv")

In [ ]:
data_train = data_train.set_index('date')
data_test = data_test.set_index('date')
data_full = data_full.set_index('date')

In [ ]:
def info(data):
    rows, columns = data.shape
    print('Количество строк:', rows)
    print('Количество столбцов:', columns)
    display(data.head(5))
    display(data.describe())
    data.info()

In [ ]:
info(data_train)

In [ ]:
info(data_test)

In [ ]:
info(data_full)

*Вывод: во всех таблицах есть пропуски. Ну и как было сказанно в описании проекта - в тестовой выборке отсутсвуют некоторые признаки из обучающей. Также, все данные имеют тип float.*

###  Проверка эффективности обогащения

In [ ]:
def recovery_calculate(c, f, t):
    recovery = (c * (f - t)) / (f * (c - t)) * 100
    return recovery

C — доля золота в концентрате после флотации/очистки;
F — доля золота в сырье/концентрате до флотации/очистки;
T — доля золота в отвальных хвостах после флотации/очистки.

In [ ]:
recovery = recovery_calculate(data_train['rougher.output.concentrate_au'],
                  data_train['rougher.input.feed_au'],
                  data_train['rougher.output.tail_au'])
mae_recovery = mean_absolute_error(data_train['rougher.output.recovery'].dropna(), recovery.dropna())
print('MAE равно:', mae_recovery)

*Вывод: средняя абсолютная ошибка между признаком и рассчитанной величиной крайне мала, из чего можно сделать вывод, что признак в таблице посчитан верно.*

### Анализ признаков, недоступных в тестовой выборке

In [ ]:
missing_columns = []
for col in range(len(data_train.columns)):
    if data_train.columns[col] not in data_test.columns:
        missing_columns.append(data_train.columns[col])
missing_columns

*Вывод: признаки с calculation скорее всего отсутствуют из-за того, что он рассчитывается значительно позже процесса. Признаки с output отсутсвуют так как они являются целевыми.*

### Предобработка данных


Посчитаем долю пропусков в столбцых таблиц.

In [ ]:
def na_perc(data):
    print((data.isna().sum() / data.shape[0] * 100)
          .sort_values(ascending=False))

In [ ]:
na_perc(data_train)

In [ ]:
na_perc(data_test)

Добавим целевые признаки в тестовую выборку из полной выборки.

In [ ]:
print(data_test.shape)
data_test = data_test.merge(data_full[TARGETS]
                            .loc[data_test.index], on='date', how='left')

print(data_test.shape)

Удалять пропуски нельзя, так как я уже пытался это делать и там уйдут 23% данных, а это много. Поэтому заполняем их. Так как по условию сказанно, что соседние значения похожи - применяем метод filna с параметром method='bfill' или 'ffill'.

In [ ]:
data_train = data_train.fillna(method='ffill')
data_test = data_test.fillna(method='ffill')

In [ ]:
print('Количество пропусков в обучающей выборке:', data_train.isna().sum().sum())
print('Количество пропусков в тестовой выборке:', data_test.isna().sum().sum())

## Анализ данных

 ### Изменение концентрации металлов (Au, Ag, Pb) на различных этапах очистки

In [ ]:
plt.figure()
data_full[['rougher.input.feed_au', 'rougher.output.concentrate_au', 
           'primary_cleaner.output.concentrate_au', 'final.output.concentrate_au']].plot(
title='Изменение концентрации Au на разных этапах очистки',
kind='hist',
grid=True,
legend=True,
bins=150,
figsize=(18, 5),
alpha=0.6)

plt.xlabel('Концентрация')
plt.axvline(data_full['rougher.input.feed_au'].mean(), ls='--', label='Средняя концентрация до флотации')
plt.axvline(data_full['rougher.output.concentrate_au'].mean(), ls='--', label='Средняя концентрация после флотации', c='m')
plt.axvline(data_full['primary_cleaner.output.concentrate_au'].mean(), ls='--', label='Средняя концентрация после первой очистки', c='g')
plt.axvline(data_full['final.output.concentrate_au'].mean(), ls='--', label='Средняя концентрация после второй очистки', c='r')

plt.legend(loc='upper right', borderaxespad= 0, bbox_to_anchor=( 1.23, 1 ))

Средняя концентрация зололта на разных этапах очистки увеличивается.

In [ ]:
data_full[['rougher.input.feed_ag', 'rougher.output.concentrate_ag', 
           'primary_cleaner.output.concentrate_ag', 'final.output.concentrate_ag']].plot(
title='Изменение концентрации Ag на разных этапах очистки',
kind='hist',
grid=True,
legend=True,
bins=150,
figsize=(18, 5),
alpha=0.6)
plt.axvline(data_full['rougher.input.feed_ag'].mean(), ls='--', label='Средняя концентрация до флотации')
plt.axvline(data_full['rougher.output.concentrate_ag'].mean(), ls='--', label='Средняя концентрация после флотации', c='m')
plt.axvline(data_full['primary_cleaner.output.concentrate_ag'].mean(), ls='--', label='Средняя концентрация после первой очистки', c='g')
plt.axvline(data_full['final.output.concentrate_ag'].mean(), ls='--', label='Средняя концентрация после второй очистки', c='r')
plt.legend(loc='upper right', borderaxespad= 0, bbox_to_anchor=( 1.23, 1 ))
plt.xlabel('Концентрация')

Концентрация серебра после этапа флотации увеличилась, а на дальнейших этапах снижалась.

In [ ]:
data_full[['rougher.input.feed_pb', 'rougher.output.concentrate_pb', 
           'primary_cleaner.output.concentrate_pb', 'final.output.concentrate_pb']].plot(
title='Изменение концентрации Pb на разных этапах очистки',
kind='hist',
grid=True,
legend=True,
bins=150,
figsize=(18, 5),
alpha=0.6)
plt.axvline(data_full['rougher.input.feed_pb'].mean(), ls='--', label='Средняя концентрация до флотации')
plt.axvline(data_full['rougher.output.concentrate_pb'].mean(), ls='--', label='Средняя концентрация после флотации', c='m')
plt.axvline(data_full['primary_cleaner.output.concentrate_pb'].mean(), ls='--', label='Средняя концентрация после первой очистки', c='g')
plt.axvline(data_full['final.output.concentrate_pb'].mean(), ls='--', label='Средняя концентрация после второй очистки', c='r')
plt.legend(loc='upper right', borderaxespad= 0, bbox_to_anchor=( 1.23, 1 ))
plt.xlabel('Концентрация')

Концентрация свинца увеличививается, но после первичной очистки увеличение очень незначительное.

### Распределения размеров гранул сырья на обучающей и тестовой выборках

In [ ]:
plt.figure(figsize=(20,5))
data_train['rougher.input.feed_size'].hist(bins=200, alpha=0.5, label='train', range=(0,150), density=True)
data_test['rougher.input.feed_size'].hist(bins=200, alpha=0.5, label='test', range=(0,150), density=True)
plt.xlabel('Размер')
plt.legend()
plt.title('Размер гранул на этапе флотации')

In [ ]:
plt.figure(figsize=(20,5))
data_train['primary_cleaner.input.feed_size'].hist(bins=200, alpha=0.5, label='train', range=(5,10), density=True)
data_test['primary_cleaner.input.feed_size'].hist(bins=200, alpha=0.5, label='test', range=(5,10), density=True)
plt.xlabel('Размер')
plt.legend()
plt.title('Размер гранул на этапе первой очситки')

Также для сравнения распределений хочу применить тест Колмогорова-Смирнова.

In [ ]:
stat, p_value = ks_2samp(data_train['rougher.input.feed_size'], data_test['rougher.input.feed_size'])
print(f'Kolmogorov-Smirnov Test: statistic={stat:.4f}, p-value={p_value}')

Значение p-value ниже 5%, поэтому отвергаем нулевую гипотезу об одинаковости двух распределений.

In [ ]:
stat, p_value = ks_2samp(data_train['primary_cleaner.input.feed_size'], data_test['primary_cleaner.input.feed_size'])
print(f'Kolmogorov-Smirnov Test: statistic={stat:.4f}, p-value={p_value}')

Вывод: распределения различаются из-за большого количества наблюдений в обучающей выборке. Но визуально распределения похожи.

### Исследование суммарной концентрации всех веществ на разных стадиях: в сырье, в черновом и финальном концентратах

#### Исследование суммарной концентрации всех веществ на стадии сырья.

In [ ]:
sum_rougher_feed = data_full[['rougher.input.feed_ag', 
                   'rougher.input.feed_pb', 
                   'rougher.input.feed_sol', 
                   'rougher.input.feed_au']].sum(axis=1)


In [ ]:
sum_rougher_feed.hist(bins=250, figsize=(20,7))
plt.title('Суммарная концентрация всех веществ в сырье')

In [ ]:
print(f'Границы нормальных значений: ({round(sum_rougher_feed.quantile(0.25) - 1.5 * iqr(sum_rougher_feed), 2)}, {round(sum_rougher_feed.quantile(0.75) + 1.5 * iqr(sum_rougher_feed), 2)})')

In [ ]:
data_train['sum_rougher_feed'] = sum_rougher_feed

#### Исследование суммарной концентрации всех веществ на чернового концентрата.

In [ ]:
sum_rougher_conc = data_full[['rougher.output.concentrate_pb', 
                   'rougher.output.concentrate_ag', 
                   'rougher.output.concentrate_sol', 
                   'rougher.output.concentrate_au']].sum(axis=1)

In [ ]:
sum_rougher_conc.hist(bins=250, figsize=(20,7))
plt.title('Суммарная концентрация всех веществ в черновом концентрате')

In [ ]:
print(f'Границы нормальных значений: ({round(sum_rougher_conc.quantile(0.25) - 1.5 * iqr(sum_rougher_conc), 2)}, {round(sum_rougher_conc.quantile(0.75) + 1.5 * iqr(sum_rougher_conc), 2)})')

In [ ]:
data_train['sum_rougher_conc'] = sum_rougher_conc

#### Исследование суммарной концентрации всех веществ на стадии финального концентрата.

In [ ]:
sum_final_conc = data_full[['final.output.concentrate_pb', 
                   'final.output.concentrate_ag', 
                   'final.output.concentrate_sol', 
                   'final.output.concentrate_au']].sum(axis=1)

In [ ]:
sum_final_conc.hist(bins=250, figsize=(20,7))
plt.title('Суммарная концентрация всех веществ в финальном концентрате')

In [ ]:
print(f'Границы нормальных значений: ({round(sum_final_conc.quantile(0.25) - 1.5 * iqr(sum_final_conc), 2)}, {round(sum_final_conc.quantile(0.75) + 1.5 * iqr(sum_final_conc), 2)})')

In [ ]:
data_train['sum_final_conc'] = sum_final_conc

Избавимся от аномалий в обучающей выборке.


In [ ]:
print(data_train.shape)
data_train = data_train.loc[(40.84 < data_train['sum_rougher_feed']) & (data_train['sum_rougher_feed'] < 74.62)]

data_train = data_train.loc[(53.47 < data_train['sum_rougher_conc']) & (data_train['sum_rougher_conc'] < 86.89)]

data_train = data_train.loc[(62.77 < data_train['sum_final_conc']) & (data_train['sum_final_conc'] < 76.53)]
print(data_train.shape)

*Вывод: Мы удалили 13.5% значений из обучающей выборки, поскольку они были аномальными.*

## Модель

Удалим признаки, которых нет в тестовой выборке, из обучающей выборки.

In [ ]:
data_train = data_train.drop(set(missing_columns) - set(TARGETS), axis='columns')
data_train.shape

### Функция для вычисления итоговой sMAPE

In [ ]:
def smape_calculate(target, predictions):
    x = np.abs(target - predictions)
    y = np.abs(target + predictions)
    return 1 / len(target) * np.sum(x / (y / 2)) *100

def final_smape_calculate(target, predictions):
    target_r = target['rougher.output.recovery']
    target_f = target['final.output.recovery']
    pred_r = predictions[:, 0]
    pred_f = predictions[:, 1]
    return 0.25 * smape_calculate(target_r, pred_r) + 0.75 * smape_calculate(target_f, pred_f)

In [ ]:
features_train = data_train.drop(TARGETS, axis='columns')
target_train = data_train[TARGETS]
features_test = data_test.drop(TARGETS, axis='columns')
target_test = data_test[TARGETS]

In [ ]:
scorer = make_scorer(final_smape_calculate, greater_is_better=False)

### Обучение моделей

#### Дерево решений

In [ ]:
params_DT= {"max_depth": range(1, 15)}

In [ ]:
state = RandomState(12345)

model_DT = RandomizedSearchCV(DecisionTreeRegressor(random_state=state), params_DT, scoring = scorer, cv=5)
model_DT.fit(features_train, target_train)
print(f'Лучшее sMape для дерева решений: {-model_DT.best_score_} при гиперпараметрах: {model_DT.best_params_}')

#### Случайный лес

In [ ]:
params_RF = {'n_estimators' : range(10, 31, 10)}

In [ ]:
for i in tqdm(range(0, 1), desc ="Готовность"):
    model_RF = RandomizedSearchCV(RandomForestRegressor(random_state=state),params_RF, cv=5, scoring=scorer)
    model_RF.fit(features_train, target_train)
print(f'Лучшее sMape для случайного леса: {-model_RF.best_score_} при гиперпараметрах: {model_RF.best_params_}')

#### Линейная регрессия

In [ ]:
model_LR = LinearRegression()
model_LR.fit(features_train, target_train)
score = cross_val_score(model_LR, features_train, target_train, cv=5, scoring=scorer)
print(f'Лучшее sMape для линейной регрессии: {-(max(score))}')

Лушчая модель -  линейная регрессия. Для проверки тестовой выборки будем использовать ее.

In [ ]:
features_train = features_train.drop(['sum_final_conc', 'sum_rougher_conc', 'sum_rougher_feed'], axis=1)

In [ ]:
model_LR = LinearRegression()
model_LR.fit(features_train, target_train)
pred = model_LR.predict(features_test)
print(f'smMAPE для тестовой выборки равна: {final_smape_calculate(target_test, pred)}')

In [ ]:
const_model = DummyRegressor(strategy = 'median')
const_model.fit(features_train, target_train)
pred = const_model.predict(features_test)
print(f'smMAPE константной модели для тестовой выборки равна: {final_smape_calculate(target_test, pred)}')

*Вывод:*

*1)в ходе исследования я провел предобработку данных;*

*2)посмотрел, как меняется концентрация металлов (Au, Ag, Pb) на различных этапах очистки;*

*3)сравнил распределения размеров гранул сырья на обучающей и тестовой выборках;*

*4)исследовал суммарную концентрацию всех веществ на разных стадиях: в сырье, в черновом и финальном концентратах;* 

*5)в конце мне было необходимо построить модель, которая предсказывает коэффициент восстановления золота из золотосодержащей руды. Из рассмотренных мной моделей лучше всех проявила себя линейная регрессия, на тестовой выборке она показала себя чуть лучше константной модели*